<a href="https://colab.research.google.com/github/CapitanMurloc/srt/blob/develop/Text_To_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/
!mkdir tts
%env HF_HOME=/content/drive/MyDrive/tts

In [1]:
!git clone https://github.com/pytorch/fairseq.git
%cd /content/fairseq
!pip install -r requirements.txt
!python setup.py build develop

Cloning into 'fairseq'...
remote: Enumerating objects: 33399, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 33399 (delta 0), reused 2 (delta 0), pack-reused 33397
Receiving objects: 100% (33399/33399), 23.28 MiB | 35.80 MiB/s, done.
Resolving deltas: 100% (24405/24405), done.
/content/fairseq
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/fairseq_cli
copying fairseq_cli/train.py -> build/lib.linux-x86_64-3.7/fairseq_cli
copying fairseq_cli/__init__.py -> build/lib.linux-x86_64-3.7/fairseq_cli
copying fairseq_cli/interactive.py -> build/lib.linux-x86_64-3.7/fairseq_cli
copying fairseq_cli/eval_lm.py -> build/lib.linux-x86_64-3.7/fairseq_cli
copying fairseq_cli/hydra_train.py -> build/lib.linux-x86_64-3.7/fairseq_cli
copying fairseq_cli/score.py -> build/lib.

In [ ]:
!pip install fairseq
!pip install huggingface-hub
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 2.7 MB/s 
     |████████████████████████████████| 118 kB 78.6 MB/s 
     |████████████████████████████████| 123 kB 86.9 MB/s 
     |████████████████████████████████| 235 kB 90.4 MB/s 
     |████████████████████████████████| 112 kB 91.5 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=deb0a999fb3760ad62632c2aa3bcb24155c3246e9bd11aa9538938b56f50293a
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████

In [ ]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator([model], cfg)

text = "En este vídeo, estudiaremos la arquitectura del codificador, un ejemplo de arquitectura popular de sólo codificador"

sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

In [ ]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.speech_to_text.hub_interface import S2THubInterface
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torchaudio


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/xm_transformer_600m-en_es-multi_domain",
    arg_overrides={"config_yaml": "config.yaml"},
)
model = models[0]
generator = task.build_generator([model], cfg)


# requires 16000Hz mono channel audio
audio, _ = torchaudio.load("/content/audio.flac")

sample = S2THubInterface.get_model_input(task, audio)
text = S2THubInterface.get_prediction(task, model, generator, sample)
print(text)

# speech synthesis
tts_models, tts_cfg, tts_task = load_model_ensemble_and_task_from_hf_hub(
  "facebook/tts_transformer-es-css10",
  arg_overrides={"vocoder": "griffin_lim", "fp16": False},
)
tts_model = tts_models[0]
TTSHubInterface.update_cfg_with_data_cfg(tts_cfg, tts_task.data_cfg)
tts_generator = tts_task.build_generator([tts_model], tts_cfg)

tts_sample = TTSHubInterface.get_model_input(tts_task, text)
wav, sr = TTSHubInterface.get_prediction(
    tts_task, tts_model, tts_generator, tts_sample
)

ipd.Audio(wav, rate=rate)


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]